# Praktikum 1 - Working with datasets

In this lecture we'll go over some basic operations we typically perform with a dataset. In particular, we'll address the following points:
- Load and preview dataset
- Clean and filter rows
- Basic stats and value counts
- Grouping and aggregation
- Visualizing class balance



In [ ]:
import pandas as pd
from datasets import load_dataset, load_from_disk

# 1 Loading datasets

### 1.1 Download HF (remote) datasets
We can use datasets by using the `.load_dataset()` function from Hugging Face's dataset module. Datasets provides loading scripts to handle the loading of local and remote datasets.

In [ ]:
ds = load_dataset("dbarbedillo/SMS_Spam_Multilingual_Collection_Dataset")

Unfortunately, this node does not have access to the Internet. We can deal with this in different ways, for example with a python script that downloads any given huggingface dataset and it to disc (e.g., download_dataset.py).


```python
import sys
from datasets import load_dataset

def main():
    if len(sys.argv) != 3:
        print("Usage: python download_dataset.py <dataset_name> <output_path>")
        print("Example: python download_and_save_dataset.py <hf-user>/<ds-name> /tmp/spam")
        sys.exit(1)

    dataset_name = sys.argv[1]
    output_path = sys.argv[2]

    print(f"Loading dataset: {dataset_name}")
    ds = load_dataset(dataset_name)

    print(f"Saving dataset to: {output_path}")
    ds.save_to_disk(output_path)

    print("Done.")

if __name__ == "__main__":
    main()
```


Run the code in the terminal:

```bash
python download_dataset.py dbarbedillo/SMS_Spam_Multilingual_Collection_Dataset data/sms_spam
```

### 1.2 Loading from disk
Now we can load the dataset we downloaded with the `.load_from_disk()` function.

In [ ]:
ds_hf = load_from_disk("data/sms_spam")

### 1.3 Loading a CSV file
You can still use the `datasets` library, or alternatively directly `pandas`. But first let's download a dataset from our terminal (due to the internet connection restriction):

```bash
curl -L -o data/sms-spam-collection-dataset.zip https://www.kaggle.com/api/v1/datasets/download/uciml/sms-spam-collection-dataset && unzip data/sms-spam-collection-dataset.zip -d data/
```

This returns a csv file: ``data/spam.csv``

#### a. Using HF datasets
The datasets library can support different data formats (see https://huggingface.co/learn/llm-course/chapter5/2)

In [ ]:
ds_kg = load_dataset("csv", data_files="data/spam.csv", encoding="latin1")

#### b. Using Pandas
We use `pd.read_csv()` to load our CSV file into a pandas DataFrame. Here we also specify the encoding as 'latin1' to handle special characters properly.

In [ ]:
df_kg = pd.read_csv("data/spam.csv", encoding="latin1")

### 1.4 Datasets vs Pandas


`Pandas` is a powerful and intuitive tool for exploratory data analysis and manipulation, especially well-suited for small to medium datasets. On the other hand, `datasets` is tailored for machine learning workflows — offering efficient storage, native support for large datasets, and tight integration with Hugging Face models.


| Feature               | `pandas.read_csv()`                     | `datasets.load_dataset("csv")`           |
|-----------------------|-----------------------------------------|-------------------------------------------|
| Format                | DataFrame (in-memory)                   | Dataset (Arrow-backed, memory-efficient) |
| Best for              | EDA, tabular ML tasks                   | NLP workflows, large datasets            |
| Integration           | Manual with Transformers                | Native (`map`, `tokenize`, `train_test_split`) |
| Performance (large)   | RAM-limited                             | Arrow streaming, efficient for big data  |
| Save/Load             | CSV, Pickle                             | `save_to_disk()`, export to CSV          |

They are also interoperable:
  ```python
  df = dataset.to_pandas()           # dataset -> pandas
  dataset = Dataset.from_pandas(df)  # pandas -> dataset
```

# 2 Exploring the dataset

### 2.1 Dataset summary
The `.info()` method gives a concise summary of the DataFrame — including column names, non-null counts, and data types.
This is useful to check for missing data or inconsistencies.

In [ ]:
df_kg.info()

### 2.2 Previewing the data
The `.head()` method shows the first few rows of the DataFrame — useful to get a quick glance at the content and format of the dataset.

In [ ]:
df_kg.head()

### 2.3 Dataset dimensions
Using `.shape` gives the number of rows and columns in the dataset in the form (rows, columns).

In [ ]:
df_kg.shape

### 2.4 Columns and indexes
`.columns` returns an index of column names in the DataFrame.
Helpful to know what fields are available and how they are labeled.


In [ ]:
df_kg.columns

In [ ]:
df_kg.index

# 3 Accessing the dataset

## 3.1 Accesing values in a column
Using `df["v1"]` accesses the `v1` column — this returns a pandas Series containing its values. You can treat it like a list or apply string operations, aggregation, etc.

In [ ]:
df_kg["v1"]

## 3.2 Slicing Data with `df.loc`
We use `df.loc[rows, columns]` to **slice the dataset** by label or condition.

- `rows` can be a **range**, a **list of indices**, or a **boolean mask**
- `columns` selects specific column(s)

Examples:

```python
df.loc[0]                  # First row (all columns)
df.loc[0:4, "v2"]        # Rows 0–4, only "text" column
df.loc[df["v1"] == "spam", ["v1", "v2"]]  # All spam messages

In [ ]:
df_kg.loc[ : , "v1" ]

In [ ]:
df_kg.loc[0:4 , "v1"]

## 3.3 Filtering rows by condition
Using .loc[...] with a condition allows us to select rows where v1 == 'spam'.
This filters the DataFrame and returns only spam-labeled messages along with their texts.

In [ ]:
df_kg.loc[df_kg["v1"] == "spam", ["v1", "v2"]]

Sometimes we need to chain operations. For example, apply more than one filter and even some manipulations in between. We can do this step by step, like below, storing the intermediate steps to perform a filter using a boolean mask:

In [ ]:
df_spam = df_kg.loc[df_kg["v1"] == "spam", ["v1", "v2"]]
df_spam.loc[ df_spam["v2"].str.len() > 150, : ]

We can also use `lambda` functions inside `.loc`, which are more flexible for **chaining** and **piping** operations.  
In this case, the parameter `d` represents the **DataFrame returned by the previous operation**.  
This makes the logic more modular and avoids creating intermediate variables.

In [ ]:
df_kg.loc[df_kg["v1"] == "spam", ["v1", "v2"]] \
     .loc[ lambda d : d["v2"].str.len() > 150, : ]

## 3.4 Notes about indexing

After filtering row, notice that we do no longer have the 1-N ordering. Above you probably see 2, 8, .. This is because when filtering by rows we are obtaining a view. This means that if we want to obtain the first row, we cannot longer just use the index `0`. Instead, we should get the index of the first element `df.index[0]`


In [ ]:
df_x = df_kg.loc[df_kg["v1"] == "spam", ["v1", "v2"]] \
     .loc[ lambda d : d["v2"].str.len() > 150, : ]

# df_x.loc[0]  # Causes a KeyError error
df_x.loc[df_x.index[0]] # Obtains the first element of the current view



# 4 Basic Manipulations

## 4.1 Views vs Copies
When we slice the dataframe and select only columns, `loc` returns a new dataset which is esentially a copy. This means that all operations on this new dataframe do not affect the original one. Let's put this to the test:

In [ ]:
df_spam = df_kg.loc[ :, ["v1", "v2"]]
df_spam["v2"] = "changed"
print(df_kg["v2"].head())  # Still original values
print(df_spam["v2"].head())  # Modified values

Instead, if we filter the dataframe by rows, the operation may raise SettingWithCopyWarning, meaning that pandas **cannot guarantee** that changes won't affect the original dataframe. 

In [ ]:
df_spam = df_kg.loc[ :, ["v1", "v2"]]

# Filter without copy
df_spam_view = df_spam[df_spam["v1"] == "spam"]

# Modify first row in-place (⚠️ may raise SettingWithCopyWarning)
df_spam_view.loc[df_spam_view.index[0], "v2"] = "CHANGED"

# View result
print(df_spam_view.head())

# Check if original is affected
print("Original df_kg:")
print(df_spam[df_spam["v1"] == "spam"].head())

Even though it might seem to work fine, it is not given and depends how pandas handles internal optmisations. The best way to safely manipulate the data in this case is creating a copy:

In [ ]:
df_spam_view = df_spam[df_spam["v1"] == "spam"].copy()
df_spam_view.loc[df_spam_view.index[0], "v2"] = "CHANGED"

## 4.2 Working with columns

### 4.2.1 Creating columns
To **create a new column** in a DataFrame, you simply assign a value or transformation to a column label that doesn’t yet exist. Pandas will automatically create it.

In [ ]:
df_spam["is_spam"] = df_spam["v1"] == "spam"
df_spam

### 4.2.2 Dropping columns
If a column is no longer needed (e.g., a temporary or duplicate column), you can **remove it** using `.drop(columns=[...])`:

In [ ]:
df_spam.drop(columns=["v1"], inplace=True)
df_spam

### 4.2.3 Renaming columns
The `.rename(columns={...})` method allows you to assign new names to existing columns.
This is useful for making column names more meaningful (e.g., renaming 'v2' to 'text').

In [ ]:
df_spam.rename(columns={"v2" : "text" }, inplace = True)
df_spam

### 4.2.4 Alternatives
You could also directly create a clean version of your dataset by selecting and renaming columns in one go. There are two common approaches:

1. **Chaining selection + rename**:  
   This is explicit and avoids relying on column order.

2. **Manual column reassignment**:  
   This works well when dynamically assigning new column names.  


In [ ]:
# Method 1: Chain selection and rename
df_clean_1 = df_kg[["v1", "v2"]].rename(columns={"v1": "label", "v2": "text"})

# Method 2: Assign column names manually
df_clean_2 = df_kg[["v1", "v2"]].copy()  # Copy is not necessary, but good defensive practice
df_clean_2.columns = ["label", "text"]

# Inspect data frame (switch between _1 and _2 to check)
df_clean_2.head()


## 4.3 Working with rows and values

### 4.3.1 Replacing values

We can replace specific values using using the `replace` function. For example, to replace labels like `"spam"` and `"ham"` with numeric values:


In [ ]:
#df["label"] = df["label"].replace({"spam": 1, "ham": 0})
df_spam.info()

### 4.3.2 Custom transformations using lambda functions
We can use `.apply()` with a lambda function to transform rows or values. This can be applied at the column level or at the dataframe level. 

In [ ]:
# We apply it to the values of column "text", the function we passed is apply to each value individually
df_spam["text"].apply(lambda t : len(t))
# df_spam["text"].apply(len).head() # Since len is a function you could pass it directly


Applying a function at the DataFrame level allows us to use values from **multiple columns** at once.

By default, `.apply()` works **column-wise** (`axis=0`).  
To process each row individually, we must specify `axis=1`. 

In [ ]:
df_spam.apply( lambda row : row["is_spam"] and len(row["text"]) > 100, axis= 1)

### 4.3.2 Sorting rows by a column
We can sort the rows in our dataframe with `.sort_values()` by one or more columns. For example, if we want to sort it by lenght of of the text.

```python
df.sort_values(by=["length", ascending=False, inplace=False)
```
`by` and `ascending` can receive a string in case of single columns, or an array in case of multiple columns. By default, it creates a new dataset unless `inplace=True`

In [ ]:
df_spam["length"] = df_spam["text"].apply(lambda t : len(t))

# Example with one column
df_sorted = df_spam.sort_values(by="length", ascending=False)

# Example with multiple columns:
#df_sorted = df_spam.sort_values(by=["label", "length"], ascending=[True, False]).head()

# Inspect sorted dataframe
df_sorted.head()


The idexes from the original dataset are preserved by default (as you can see above). This is to maintain reference i can you want to later join / merge different dataframes or even backtracking. To reindex the dataframe, run `reset_index` with `drop=True` if you don't want the old index to be added as a new column.

In [ ]:
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

# df_sorted.reset_index() # -> this would add a new column "index" with the old index

# You can chain reset index with sort_values
# df_sorted = df_spam.sort_values(by="length", ascending=False).reset_index(drop=True)



### 4.3.3 Dropping values
We can drop values using different methods, as shown in the Table below:

| Use Case                          | Method / Code Example                                         | Notes |
|----------------------------------|---------------------------------------------------------------|-------|
| ❌ Drop row by index              | `df.drop(index=42)`                                           | Drops row with index label `42` |
| ❌ Drop multiple rows by index    | `df.drop(index=[0, 3, 5])`                                    | Drops several rows |
| ❌ Drop rows with condition       | `df = df[df["label"] != "spam"]`                             | Keeps only rows where condition is True |
| ❌ Drop rows with missing values  | `df.dropna(subset=["text"])`                                 | Only drops if `"text"` is missing |
| ❌ Drop duplicates by column      | `df.drop_duplicates(subset=["text"])`                        | Removes repeated messages |
| Reset index after dropping     | `df.reset_index(drop=True, inplace=True)`                    | Keeps the index clean |


When cleaning data, we often need to **remove rows that match certain conditions** — for example, empty texts or unwanted labels. A flexible and common approach in Pandas is to **create a new DataFrame by selecting only the rows we want to keep**. We do this using **Boolean indexing**.
How it works:

- We define a **condition** that returns a **Boolean Series** — one `True` or `False` for each row.
- This Boolean Series must have the **same length as the DataFrame**.
- We pass it inside square brackets `df[...]` to filter the rows.
- The result is a DataFrame containing only the `True` rows.

For example, to remove empty or whitespace-only messages we would do:

In [ ]:
# Condition: df_spam["text"].str.strip() != ""
# DataFrame: df_spam

df_spam = df_spam[df_spam["text"].str.strip() != ""].reset_index(drop=True)

df_spam

# 5 Basic statistics

## 5.1 Class distributions 
Check how many examples are labeled per class. We can have raw **value counts** of proportions (normalized). For example, we want to know the distribution of spam and ham labels.



In [ ]:
df_spam["is_spam"].value_counts()

In [ ]:
df_spam["is_spam"].value_counts(normalize=True)

We can visualise this with `matplotlib`

In [ ]:
import matplotlib.pyplot as plt

df_spam["is_spam"].value_counts().plot(kind="bar", title="Class Distribution")
plt.xlabel("Is_Spam?")
plt.ylabel("Count")
plt.show()

## 5.2 Descriptive statistics (spread and centrality)
The `.describe()` method summarizes numerical columns — including count, mean, std, min, and percentiles.

In [ ]:
df_spam["length"].describe()

In [ ]:
df_spam["length"].hist(bins=30)
plt.xlabel("Message Length")
plt.ylabel("Frequency")
plt.title("Distribution of Message Lengths")
plt.show()

## 5.3 Unique, duplicate and missing values
We can list the unique values with `.unique()` and number of unique values with `.nunique()`.

In [ ]:
df_spam["text"].nunique()

### 5.3.2 Duplicates
The `.duplicated()` function in Pandas helps identify rows that are repeated in a DataFrame. It returns a Boolean Series where each value is `True` if the row is a duplicate of a **previous** one (based on all or selected columns), and `False` otherwise. By default, it keeps the **first occurrence** and marks subsequent duplicates as `True`, but you can change this behavior using the `keep` parameter (e.g., `"last"` or `False` to mark all duplicates). This is useful for spotting and removing repeated messages — a common issue in real-world NLP datasets.

In [ ]:
df_spam.duplicated(keep=False).sum()

The `.drop_duplicates()` method removes duplicated rows, keeping only the first (by default).
You can specify subset to check for duplicates based on specific columns (e.g., "text").

In [ ]:
df_spam.drop_duplicates()

### 5.3.3 Missing values
The `.isna()` method returns a Boolean DataFrame indicating which values are missing (NaN). You can use to spot those rows, count how many there are, and also delete them.

In [ ]:
df_spam.isna().sum()

### 5.3.4 Group values 
The `.groupby()` function lets you split your dataset into **groups** based on a column (e.g., labels), and then apply an **aggregation** (like mean, count, or max) to each group. This is useful when you want to compare statistics **between categories** — like spam vs ham messages.

In [ ]:
df_spam.groupby("is_spam").size()

In [ ]:
df_spam.groupby("is_spam")["length"].mean()

In [ ]:
df_spam.groupby("is_spam")["length"].agg(["mean", "max", "min", "count"])

# Additional resources

- https://github.com/TirendazAcademy/PANDAS-TUTORIAL